Learn to Determine, Classify the Score (pass/fail) for a Jannah.io Boot Deployment Using Log Files

In [ ]:
#papermill_description=Import_Python_Libraries
import pathlib
import random
import datetime
import time

import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [ ]:
#papermill_description=Determine_Directory_Paths_For_Sorting_Log_Files
Jannah_Config = provisioner["inventory"]["group_vars"]["all"]["Jannah"]
Work_Dir = provisioner["inventory"]["group_vars"]["all"]["Jannah"]['global']['ansible']['working_dir']
MOLECULE_EPHEMERAL_DIRECTORY = provisioner["env"]["MOLECULE_EPHEMERAL_DIRECTORY"]
Logs_input = f"{MOLECULE_EPHEMERAL_DIRECTORY}/logs"
Logs_train_input = f"{Logs_input}/train"
Logs_test_input = f"{Logs_input}/test"
Logs_train_input_pass_deployment = f"{Logs_train_input}/pass"
Logs_train_input_fail_deployment = f"{Logs_train_input}/fail"
Logs_test_input_pass_deployment = f"{Logs_test_input}/pass"
Logs_test_input_fail_deployment = f"{Logs_test_input}/fail"
sorted_deployment_log_paths = {
    "train": {
                "pass": Logs_train_input_pass_deployment,
                "fail": Logs_train_input_fail_deployment
    },
    "test": {
            "pass": Logs_test_input_pass_deployment,
            "fail": Logs_test_input_fail_deployment
    }
}

Run Kind Cluster Matrix

In [ ]:
#papermill_description=Generate_Log_File_Name
current_datetime = datetime.datetime.fromtimestamp(time.time()).isoformat().replace(":","-").replace(".","-")
# Logs_input="/Users/osmanjalloh/working/debug/operator/tmp/EPHEMERAL/logs"
# Work_Dir = "/Users/osmanjalloh/working/debug/operator"
deploy_mode = "deploy-to-kind-cluster-full-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
print(dir())
print(log_file)

In [ ]:
#papermill_description=make_deploy_to_kind_cluster_full_ubuntu_dev_mode
!pushd $Work_Dir && make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=deploy_to_kind_cluster_local_ubuntu_dev_mode
deploy_mode = "deploy-to-kind-cluster-local-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
print(log_file)
!pushd $Work_Dir && make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=deploy_to_kind_cluster_standalone_ubuntu_dev_mode
deploy_mode = "deploy-to-kind-cluster-standalone-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
print(log_file)
!pushd $Work_Dir && make $deploy_mode >> $log_file && popd

Run Docker Desktop Matrix

In [ ]:
#papermill_description=deploy_to_docker_desktop_full_ubuntu_dev_mode
deploy_mode = "deploy-to-docker-desktop-full-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
print(log_file)
!pushd $Work_Dir && make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=deploy_to_docker_desktop_local_ubuntu_dev_mode
deploy_mode = "deploy-to-docker-desktop-local-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
print(log_file)
!pushd $Work_Dir && make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=deploy_to_docker_desktop_local_ubuntu_dev_mode
deploy_mode = "deploy-to-docker-desktop-local-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
print(log_file)
!pushd $Work_Dir && make deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=deploy_to_docker_desktop_standalone_ubuntu_dev_mode
deploy_mode = "deploy-to-docker-desktop-standalone-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
print(log_file)
!pushd $Work_Dir && make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=jannah_deployment_with_ansible
deploy_mode = "jannah-deployment-with-ansible"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
print(log_file)
!pushd $Work_Dir && make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=Create_Directories_For_Sorting_Log_Files
!mkdir -vp $Logs_train_input_pass_deployment/
!mkdir -vp $Logs_train_input_fail_deployment/
!mkdir -vp $Logs_test_input_pass_deployment/
!mkdir -vp $Logs_test_input_fail_deployment

In [ ]:
#papermill_description=Copy_Log_Files_For_Processing
!cp -rp ~/jannah-operator/*.log $Logs_input/

In [ ]:
#papermill_description=Glob_Log_Files
log_files  = pathlib.Path(Logs_input).glob("*.log")

In [ ]:
#TODO: Note: To increase the difficulty of the classification problem, th replaced occurrences of the phrases "All assertions passed", "Assertion failed" from buffer
#      and write buffer back to file.

In [ ]:
#papermill_description=Determine_Pass_or_Fail_Status_For_Each_Deployment_File
_FILE_BUF = ""
pass_deployments = []
failed_deployments = []
for _file in sorted((log_files)):
    with _file.open() as fd:
        _FILE_BUF = fd.read()
        if "All assertions passed" in _FILE_BUF and "Assertion failed" not in _FILE_BUF:
            pass_deployments.append(_file)
        else:
            failed_deployments.append(_file)

In [ ]:
#papermill_description=Print_Pass_Deployment_Counts
len(pass_deployments)

In [ ]:
#papermill_description=Print_Failed_Deployment_Counts
len(failed_deployments)

In [ ]:
#papermill_description=Sort_Pass_Deployments_Into_Train_or_Test_Directories
import random
for _file in pass_deployments:
    train_or_test_group = random.choices(population=['train','test'], weights=[80,20]).pop()
    _new_file_name = _file.name.replace(".log",".txt")
    _new_file = f"{sorted_deployment_log_paths[train_or_test_group]['pass']}/{_new_file_name}"
    _file.rename(_new_file)

In [ ]:
#papermill_description=Sort_Fail_Deployments_Into_Train_or_Test_Directories
import random
for _file in failed_deployments:
    train_or_test_group = random.choices(population=['train','test'], weights=[80,20]).pop()
    _new_file_name = _file.name.replace(".log",".txt")
    _new_file = f"{sorted_deployment_log_paths[train_or_test_group]['fail']}/{_new_file_name}"
    _file.rename(_new_file)

In [ ]:
#papermill_description=Text_Dataset_From_Directory
train_dir=Logs_train_input
batch_size = 32
seed = 80

raw_train_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

In [ ]:
#papermill_description=Print_a_Sample_Line
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    print("Log Line: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])

In [ ]:
#papermill_description=Print_the_Labels
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

In [ ]:
#papermill_description=Create_a_validation_set.
raw_val_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

In [ ]:
#papermill_description=Create_a_test_set
test_dir = Logs_test_input
raw_test_ds = utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)

In [ ]:
#papermill_description=Configure_the_datasets_for_performance
raw_train_ds = raw_train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
raw_val_ds = raw_val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
raw_test_ds = raw_test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)


First, use the `'binary'` vectorization mode to build a bag-of-words model. Then use the `'int'` mode with a 1D ConvNet.

In [ ]:
#papermill_description=Vectorization_mode_to_build_a_bag_of_words_model
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

For the `'int'` mode, in addition to maximum vocabulary size, you need to set an explicit maximum sequence length (`MAX_SEQUENCE_LENGTH`), which will cause the layer to pad or truncate sequences to exactly `output_sequence_length` values:

In [ ]:
#papermill_description=Vectorization_mode_to_build_a_bag_of_words_model
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

Next, call `TextVectorization.adapt` to fit the state of the preprocessing layer to the dataset. This will cause the model to build an index of strings to integers.

Note: It's important to only use your training data when calling `TextVectorization.adapt`, as using the test set would leak information.

In [ ]:
#papermill_description=Make_a_text_only_dataset_without_labels_then_call_TextVectorization_adapt
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

Print the result of using these layers to preprocess data:

In [ ]:
#papermill_description=Retrieve_a_batch__of_32_log_lines_and_labels_from_the_dataset.
text_batch, label_batch = next(iter(raw_train_ds))
first_log_line, first_label = text_batch[0], label_batch[0]
print("Log Line:", first_log_line)
print("Label:", first_label)

The binary vectorization layer returns a multi-hot vector, with a 1 in the location for each token that was in the input string.

In [ ]:
#papermill_description=Print_binary_vectorized_log_lines.
print("'binary' vectorized Log lines:",
      list(binary_vectorize_layer(first_log_line).numpy()))

plt.plot(binary_vectorize_layer(first_log_line).numpy())
plt.xlim(0,1000)

In [ ]:
#papermill_description=Print_int_vectorized_log_lines.
print("'int' vectorized question:",
      int_vectorize_layer(first_log_line).numpy())

In [ ]:
#papermill_description=Print_binary_vectorize_layer_int_vectorize_layer
print("binary_vectorize_layer[0:3] ---> ", binary_vectorize_layer.get_vocabulary()[:3])
print("int_vectorize_layer[0:3] ---> ", int_vectorize_layer.get_vocabulary()[:3])
print("binary_vectorize_layer size: {}".format(len(binary_vectorize_layer.get_vocabulary())))
print("int_vectorize_layer size: {}".format(len(int_vectorize_layer.get_vocabulary())))

### Train the model

It's time to create your neural network.

For the `'binary'` vectorized data, define a simple bag-of-words linear model, then configure and train it:

In [ ]:
#papermill_description=Train_the_binary_model
binary_model = tf.keras.Sequential([
    binary_vectorize_layer,
    layers.Dense(4)])

binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

tf.keras.utils.plot_model(binary_model, show_shapes=True)

In [ ]:
#papermill_description=Train_the_binary_model_bin_history_fit
bin_history = binary_model.fit(
    raw_train_ds, validation_data=raw_val_ds, epochs=18)

print()

Next, you will use the `'int'` vectorized layer to build a 1D ConvNet:


In [ ]:
#papermill_description=def_create_model
def create_model(vocab_size, num_labels, vectorizer=None):
  my_layers =[]
  if vectorizer is not None:
    my_layers = [vectorizer]

  my_layers.extend([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Dropout(0.5),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])

  model = tf.keras.Sequential(my_layers)
  return model

In [ ]:
#papermill_description=int_model_create
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4, vectorizer=int_vectorize_layer)

tf.keras.utils.plot_model(int_model, show_shapes=True)

In [ ]:
#papermill_description=int_model_compile
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
int_history = int_model.fit(raw_train_ds, validation_data=raw_val_ds, epochs=100)

In [ ]:
#papermill_description=int_model_compile
loss = plt.plot(bin_history.epoch, bin_history.history['loss'], label='bin-loss')
plt.plot(bin_history.epoch, bin_history.history['val_loss'], '--', color=loss[0].get_color(), label='bin-val_loss')

loss = plt.plot(int_history.epoch, int_history.history['loss'], label='int-loss')
plt.plot(int_history.epoch, int_history.history['val_loss'], '--', color=loss[0].get_color(), label='int-val_loss')

plt.legend()
plt.xlabel('Epoch')
plt.ylabel('CE/token')

You are nearly ready to train your model.

As a final preprocessing step, you will apply the `TextVectorization` layers you created earlier to the training, validation, and test sets:

In [ ]:
#papermill_description=int_model_compile
binary_train_ds = raw_train_ds.map(lambda x,y: (binary_vectorize_layer(x), y))
binary_val_ds = raw_val_ds.map(lambda x,y: (binary_vectorize_layer(x), y))
binary_test_ds = raw_test_ds.map(lambda x,y: (binary_vectorize_layer(x), y))

int_train_ds = raw_train_ds.map(lambda x,y: (int_vectorize_layer(x), y))
int_val_ds = raw_val_ds.map(lambda x,y: (int_vectorize_layer(x), y))
int_test_ds = raw_test_ds.map(lambda x,y: (int_vectorize_layer(x), y))

### Export the model

In [ ]:
#papermill_description=export_bin_tf
binary_model.export('bin.tf')

In [ ]:
#papermill_description=saved_model
loaded = tf.saved_model.load('bin.tf')

In [ ]:
#papermill_description=predict
binary_model.predict(['2023-10-20 16:32:43,374 p=11445 u=osmanjalloh n=ansible | localhost                  : ok=21   changed=4    unreachable=0    failed=0    skipped=3    rescued=0    ignored=0'])

In [ ]:
#papermill_description=serve
loaded.serve(tf.constant(['localhost                  : ok=19   changed=3    unreachable=0    failed=0    skipped=9    rescued=0    ignored=0'])).numpy()